<a href="https://colab.research.google.com/github/harditrivedi16/Image-Classification--258/blob/main/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the VGG 16 Model


## Importing and Installing all the libraries

In [1]:
import os
#import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
#from torchvision.models import vgg16
import torch.optim as optim
import torch.nn as nn

import torch
#import torch.nn as nn
#import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F

## Adding the relevant class of VGG 16 model from the given github [link](https://github.com/lkk688/MultiModalClassifier/blob/main/TorchClassifier/myTorchModels/TorchCNNmodels.py)

## Mounting the drive and getting the dataset folder's path

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the path to the dataset folder
data_path = '/content/drive/MyDrive/CMPE 258 Assignment 2/Data_with_class'

### Verifying that the number of images is more than 500

In [4]:
# Function to count the number of .jpg files in the folder
def count_images(folder_path):
    jpg_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    return len(jpg_files)

In [5]:
# Count the number of images in the folder
num_images = count_images(data_path)
print("Number of images in the folder:", num_images)

Number of images in the folder: 0


## Data Preprocessing

In [6]:
# Load the dataset with ImageFolder
full_dataset = datasets.ImageFolder(data_path)

### Splitting the dataset into train and val model

In [7]:
# Calculate sizes for splitting the dataset into train and validation
train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size

# Split the dataset
train_dataset, valid_dataset = random_split(full_dataset, [train_size, valid_size])

### Applying the transform function

In [8]:
# 3. Define the transform function
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [9]:
train_dataset.dataset.transform = train_transforms
valid_dataset.dataset.transform = valid_transforms

### Define the data loaders

In [10]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

## Model Setup

In [12]:
num_classes = len(train_dataset.class)
model = models.vgg16(pretrained=True)

SyntaxError: invalid syntax (<ipython-input-12-9efa6e0c7ce8>, line 1)

In [ ]:
# Freeze all layers in the feature extractor
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
# Replace the last layer
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, num_classes)

In [ ]:
# Move the model to GPU if available
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)

## Training and validation Loop

In [ ]:
def train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=25, device='cpu'):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Train Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

        # Validation step
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(valid_loader.dataset)
        val_acc = val_corrects.double() / len(valid_loader.dataset)

        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


        #if val_acc > best_acc:
            #best_acc = val_acc

            # torch.save(model.state_dict(), 'best_model.pth')
            #print('Validation accuracy increased, saving model')

    return model


In [ ]:
num_epochs = 10  # Set the number of epochs
model_trained = train_model(model, criterion, optimizer, num_epochs=num_epochs)